# ***Name: Ahmed Nasser Ahmed Hassan***
> **CodeClause |> *Sep/2022***
>> **Data Science Intern |> *CC-OL-911***
>>> **Task3 >> *ChatBot for CodeClause***
---

> ### |> ***Import Packages***

In [1]:
import numpy as np
import json
import random
import pickle

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.models import load_model

from keras.optimizers import SGD

import tkinter
from tkinter import *

[nltk_data] Downloading package punkt to C:\Users\Ahmed
[nltk_data]     Nasser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ahmed
[nltk_data]     Nasser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Ahmed
[nltk_data]     Nasser\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


> ### |> ***Load "Intents" Json***

In [2]:
intents_JsonFile = '''
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hi there",
        "How are you",
        "Is anyone there?",
        "Hey",
        "Hola",
        "Hello",
        "Good day"
      ],
      "responses": [
        "Hello, thanks for asking",
        "Good to see you again",
        "Hi there, how can I help?"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "See you later",
        "Goodbye",
        "Nice chatting to you, bye",
        "Till next time"
      ],
      "responses": [
        "See you!",
        "Have a nice day",
        "Bye! Come back again soon."
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "thanks",
      "patterns": [
        "Thanks",
        "Thank you",
        "That's helpful",
        "Awesome, thanks",
        "Thanks for helping me"
      ],
      "responses": [
        "Happy to help!",
        "Any time!",
        "My pleasure"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "noanswer",
      "patterns": [],
      "responses": [
        "Sorry, can't understand you",
        "Please give me more info",
        "Not sure I understand"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "codeclause",
      "patterns": [
        "About codeclause",
        "What is codeclause?",
        "Services offered",
        "What is codeclause internship?",
        "codeclause"
      ],
      "responses": [
        "We offer reliable, efficient delivery with high-caliber engineers & finely-tuned software development processes.We Believe In Leadership to lead the technology to build a better future Integrity to follow truth and be real Accountability for our every commitment. We Imagine & We Engineer & We Modernize & We Manage. We also provide internships in various domains, checkout our website codeclause.com for more info",
        "At CodeClause, we endeavor to provide a formidable platform for professionals who will be our change agents - catalysts who will be torchbearers of CodeClause’s fortitude to excel and redefine limits of technology to make our clients successful. We also provide internships in various domains, checkout our website codeclause.com for more info"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "options",
      "patterns": [
        "How you could help me?",
        "What you can do?",
        "What help you provide?",
        "How you can be helpful?",
        "What support is offered"
      ],
      "responses": [
        "I can give you info about the services and internships provided by codeclause.com",
        "We can provide you internships in various domains, We have internships in -> Data Analytics & Automation AI & Internet of Things (IoT) & Software Development & Web Development & App Development",
        "Offering internships and services in -> Data Analytics & Automation AI & Internet of Things (IoT) & Software Development & Web Development & App Development"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "Internship",
      "patterns": [
        "How to apply for the internship?",
        "Where to apply for internship?",
        "Internships offered"
      ],
      "responses": [
        "Please visit the website at https://codeclause.com/careers.html to apply for internships. We have internships in -> Data Analytics & Automation AI & Internet of Things (IoT) & Software Development & Web Development & App Development"
      ],
      "context": [
        ""
      ]
    },
    {
      "tag": "offer_letter",
      "patterns": [
        "Applied for internship",
        "I have applied for internship",
        "When will I receieve my offer letter?",
        "offer letter not recieved"
      ],
      "responses": [
        "You will get your offer letter on mail around starting week of the month for which you've applied for the internship"
      ],
      "context": [
        ""
      ]
    }
  ]
}
'''

In [3]:
with open("intents.json", "w") as outfile:
  outfile.write(intents_JsonFile)

intents = json.loads(open('intents.json').read())

---

> ### |> ***Processing Dictionaries***

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [WordNetLemmatizer().lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [5]:
print(len(words), ' Word (uniques)\n', words, '\n\n')
print(len(classes), ' Class (tags)\n', classes, '\n\n')
print(len(documents), ' Document (coubles)\n', documents)

59  Word (uniques)
 ["'s", ',', 'about', 'anyone', 'applied', 'apply', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'codeclause', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'internship', 'is', 'later', 'letter', 'me', 'my', 'next', 'nice', 'not', 'offer', 'offered', 'provide', 'receieve', 'recieved', 'see', 'service', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'what', 'when', 'where', 'will', 'you'] 


7  Class (tags)
 ['Internship', 'codeclause', 'goodbye', 'greeting', 'offer_letter', 'options', 'thanks'] 


34  Document (coubles)
 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you',

In [6]:
output_empty = [0] *len(classes)
training = []

for doc in documents:
    bag = []
    pattern_words = doc[0]

    for word in pattern_words:
      pattern_words = WordNetLemmatizer().lemmatize(word.lower())

    for w in words:
      if w in pattern_words:
        bag.append(1)
      else:
        bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [7]:
random.shuffle(training)
training = np.array(training)
training

C:\Users\Ahmed Nasser\AppData\Local\Temp\ipykernel_35144\1171096522.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
        list([0, 0, 0, 0, 0, 0, 1])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 1, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([1, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 1, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

---

> ### |> ***Train Model***
      Neural Network consists of (3 layers)

In [8]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history = model.fit(np.array(train_x), np.array(train_y), epochs=250, batch_size=3)

Epoch 1/250
12/12 [==============================] - 1s 2ms/step - loss: 1.9991 - accuracy: 0.0588
Epoch 2/250
12/12 [==============================] - 0s 2ms/step - loss: 1.8743 - accuracy: 0.2941
Epoch 3/250
12/12 [==============================] - 0s 2ms/step - loss: 1.8782 - accuracy: 0.2353
Epoch 4/250
12/12 [==============================] - 0s 2ms/step - loss: 1.7782 - accuracy: 0.2941
Epoch 5/250
12/12 [==============================] - 0s 2ms/step - loss: 1.7231 - accuracy: 0.3824
Epoch 6/250
12/12 [==============================] - 0s 2ms/step - loss: 1.6875 - accuracy: 0.4412
Epoch 7/250
12/12 [==============================] - 0s 2ms/step - loss: 1.7076 - accuracy: 0.4118
Epoch 8/250
12/12 [==============================] - 0s 3ms/step - loss: 1.5722 - accuracy: 0.4412
Epoch 9/250
12/12 [==============================] - 0s 2ms/step - loss: 1.4537 - accuracy: 0.4412
Epoch 10/250
12/12 [==============================] - 0s 2ms/step - loss: 1.4893 - accuracy: 0.5294
Epoch 11/

12/12 [==============================] - 0s 2ms/step - loss: 0.6577 - accuracy: 0.7353
Epoch 84/250
12/12 [==============================] - 0s 2ms/step - loss: 0.7201 - accuracy: 0.6765
Epoch 85/250
12/12 [==============================] - 0s 1ms/step - loss: 0.7329 - accuracy: 0.6471
Epoch 86/250
12/12 [==============================] - 0s 2ms/step - loss: 0.7088 - accuracy: 0.7353
Epoch 87/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6583 - accuracy: 0.7353
Epoch 88/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6423 - accuracy: 0.7647
Epoch 89/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6736 - accuracy: 0.7353
Epoch 90/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6420 - accuracy: 0.7353
Epoch 91/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6299 - accuracy: 0.6765
Epoch 92/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6612 - accuracy: 0.6765
Epoch 93/250


12/12 [==============================] - 0s 2ms/step - loss: 0.6354 - accuracy: 0.7059
Epoch 165/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6222 - accuracy: 0.7353
Epoch 166/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6455 - accuracy: 0.7353
Epoch 167/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6220 - accuracy: 0.7059
Epoch 168/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6405 - accuracy: 0.7059
Epoch 169/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6333 - accuracy: 0.7353
Epoch 170/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5975 - accuracy: 0.7353
Epoch 171/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6578 - accuracy: 0.7059
Epoch 172/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6135 - accuracy: 0.7647
Epoch 173/250
12/12 [==============================] - 0s 1ms/step - loss: 0.6661 - accuracy: 0.7059
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.5868 - accuracy: 0.7353
Epoch 246/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6262 - accuracy: 0.7059
Epoch 247/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6887 - accuracy: 0.6765
Epoch 248/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6139 - accuracy: 0.7059
Epoch 249/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6137 - accuracy: 0.7353
Epoch 250/250
12/12 [==============================] - 0s 1ms/step - loss: 0.6561 - accuracy: 0.6765


In [11]:
model.save('chatbot-model.h5', history)
print('Model \"chatbot-model.h5\" Saved')

Model "chatbot-model.h5" Saved


---

> ### |> ***Predict Responses***

In [12]:
model = load_model('chatbot-model.h5')
print('Model \"chatbot-model.h5\" Loaded')

Model "chatbot-model.h5" Loaded


In [13]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [WordNetLemmatizer().lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [14]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag=[0] *len(words)

  for s in sentence_words:
      for i, w in enumerate(words):
          if w == s:
              bag[i]=1
              if show_details:
                print("found in bag: %s" % w)
  return (np.array(bag))

In [15]:
def predict_class(sentence, model):
  return_list = []
  p = bow(sentence, words, show_details = False)
  res = model.predict(np.array([p]))[0]
  error_threshhold = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > error_threshhold]
  results.sort(key = lambda x:x[1], reverse=True)

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list

In [16]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
      if(i['tag'] == tag):
          result = random.choice(i['responses'])
          break
  return result

In [17]:
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

---

> ### |> ***Graphical User Interface (GUI)***

In [18]:
def send():
  msg = EntryBox.get("1.0", 'end-1c').strip()
  EntryBox.delete("0.0", END)

  if msg != '':
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You|> " + msg + '\n\n')
    ChatLog.config(foreground="#FDD428", font=("Helvetica", 12, 'bold' ))

    res = chatbot_response(msg)
    ChatLog.insert(END, "Codeclause|>  " + res + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

In [19]:
base = Tk()
base.title("ChatBot-CodeClause")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="#002E5B",height=10, width=100, font="Arial",)
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(
    base, bd=1,
    font=("Verdana", 12, 'bold'),
    text="Send", command=send,
    width=10, height=3,
    bg="#FDD428", fg='#000000',
    activebackground="#ececec"
    )

EntryBox = Text(
    base, bd=2,
    font="Arial",
    width=29, height=3,
    bg="white",
    )

scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(relheight=0.100, relwidth=0.77, relx=0.01, rely=0.80)
SendButton.place(relx=0.79, rely=0.80, relheight=0.100, relwidth=0.20)

base.mainloop()

1/1 [==============================] - 0s 20ms/step
